<a href="https://colab.research.google.com/github/Viktory4121/MORD/blob/main/%D0%94%D0%9E%D0%9C.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Динамические окрестностные модели (ДОМ)
####Вариант 5

In [119]:
import numpy as np
import pandas as pd
import random
import copy
import math
from prettytable import PrettyTable
import itertools as it

In [ ]:
#A={a1, a2, a3}

#какие дуги входят в позицию:
#Ox[a1]={a1}
#Ox[a2]={}
#Ox[a3]={a1, a2}

#в какие позиции входят v
#Ov[a1]={a1}
#Ov[a2]={a2}
#Ov[a3]={a3}

In [ ]:
#количества состояний/входов/выходов
count = [3, 3, 3]
Sx = np.array([[1, 0, 1], [0, 0, 1], [0, 0, 0]])
Sv = np.array([[1, 0, 0], [0, 1, 0], [0, 0, 1]])

print("Матрица смежности:")
print("Sx = ")
print(Sx)
print("Sv = ")
print(Sv)

Матрица смежности:
Sx = 
[[1 0 1]
 [0 0 1]
 [0 0 0]]
Sv = 
[[1 0 0]
 [0 1 0]
 [0 0 1]]


In [120]:
def x1(x, v):
  return math.sin(math.log(abs(v[1] + v[0])) / x[2])

def x2(x, v):
  return math.cos(v[0] * x[2] - (x[1] / x[2])**3)

def x3(x, v):
  return math.sin(math.tan(v[0] / v[2]) - 3**(x[2]))

def v1(x, v):
  return (x[1] * v[0] * v[1] + math.tan(x[2])**2)

def v2(x, v):
  return (0.2 * v[0] + 33.0 * v[1] + v[2])

def v3(x, v):
  return (math.exp(x[0]) + .45 * math.log2(abs(x[1] * x[2])))

def y1(x, v):
  return (x[0] / v[0]) * math.sqrt(abs(x[0] / x[1]))

def y2(x, v):
  return math.sqrt(abs(math.sin(v[2])))

def y3(x, v):
  return v[2] * v[0] / v[0]

functions = [x1, x2, x3, v1, v2, v3, y1, y2, y3]

In [ ]:
#Метод псевдообращения Бена-Израиля
def BenIsrael(A, eps):
  I = np.diag(np.diag(np.ones([len(A),len(A)])))
  buf = 1
  i = 1

  if(np.linalg.norm(A, ord='fro') != 0): #если не нулевая норма
    gamma = 1.6 / (np.linalg.norm(A, ord='fro')**2)
    A1 = gamma * (A.T)
    i += 1
    A2 = A1.dot(2*I - A.dot(A1))
    E = np.linalg.norm(A2 - A1, ord=np.inf) / np.linalg.norm(A1, ord=np.inf)
  else: #если нулевая норма
    A1 = np.zeros((len(A[0]), len(A)))
    buf = 0

  while(E >= eps and buf):
    A1 = A2.copy()
    A2 = A1.dot(2*I - A.dot(A1))
    E = np.linalg.norm(A2 - A1, ord=np.inf) / np.linalg.norm(A1, ord=np.inf)
    i += 1

  return A1

#=================================================================
#Генерация случайного вектора из 3-х элементов в диапазоне от -3 до 3
def generationRandomVector(elem = 3, start = -3, end = 3):
  vector = []
  for i in range(elem):
    vector.append(random.uniform(start, end))
  return vector

#=================================================================
#создание столбца таблицы
def createVariants(function, x, v, typeFunc = 1, elem = 100):
  vector = []

  if(typeFunc == 0):
    for i in range(elem):
      vector.append(function(generationRandomVector(), generationRandomVector()))
  else:
    for i in range(elem):
      vector.append(function(x[i], v[i]))

  return vector

#=================================================================
#count(x, v, y)
def createTableDOM(functions, count, step, oldTable = []):
  table = []
  vector = []
  columns = createColumns(step, count)

  if(step == 0):
    for i in range(sum(count[0:2])):
      table.append(createVariants(functions[i], [], [], 0)) #xi, vi

  else:
    for i in range(count[0]): #xi
      table.append(createVariants(functions[i], (np.array(oldTable))[:, 0:count[0]], (np.array(oldTable))[:, count[0]:sum(count[0:2])]))
    for i in range(count[1]): #vi
      table.append(createVariants(functions[count[0] + i], [], [], 0))

  for i in range(count[2]): #yi
    table.append(createVariants(functions[sum(count[0:2]) + i], (np.array(table).T)[:, 0:count[0]], (np.array(table).T)[:, count[0]:sum(count[0:2])]))

  table = pd.DataFrame(np.array(table).T, columns = columns)
  return table

def createColumns(step, count):
  columns = []
  for i in range(count[0]):
    columns.append("x" + str(i + 1) + "(" + str(step) + ")")
  for i in range(count[1]):
    columns.append("v" + str(i + 1) + "(" + str(step) + ")")
  for i in range(count[2]):
    columns.append("y" + str(i + 1) + "(" + str(step) + ")")
  return columns

In [ ]:
#functions = [x1, x2, x3, v1, v2, v3, y1, y2, y3]
model_0 = createTableDOM(functions, count, 0)
model_0

,x1(0),x2(0),x3(0),v1(0),v2(0),v3(0),y1(0),y2(0),y3(0)
0,-0.936341,0.409681,0.993865,2.334547,100.727537,14.171705,-0.606353,0.999702,14.171705
1,-0.415877,0.717111,-0.559389,5.520247,-5.439362,1.198534,-0.057371,0.965146,1.198534
2,0.287635,-1.000000,-0.809394,6.911150,-8.953516,1.307042,0.022321,0.982557,1.307042
3,0.929539,-0.782032,-0.960408,5.715093,52.567438,5.664534,0.177323,0.761536,5.664534
4,0.996629,0.003083,-0.535934,-0.278063,-44.968645,0.505195,-64.442562,0.695685,0.505195
...,...,...,...,...,...,...,...,...,...
95,-0.999365,-0.620762,0.887809,18.521173,49.840010,6.599388,-0.068463,0.557638,6.599388
96,0.795843,0.932346,0.997977,6.884324,-72.162667,-0.416482,0.106805,0.636039,-0.416482
97,0.333155,-0.996016,0.999062,0.361718,-32.738212,-0.164800,0.532681,0.405037,-0.164800
98,0.916318,-0.575175,0.877329,7.692509,29.376659,2.648975,0.150349,0.687702,2.648975


In [ ]:
model_1 = createTableDOM(functions, count, 1, np.array(model_0))
model_1

,x1(1),x2(1),x3(1),v1(1),v2(1),v3(1),y1(1),y2(1),y3(1)
0,-0.676135,-0.628316,-0.322137,1.011740,46.327546,-0.920823,-0.693254,0.892244,-0.920823
1,-0.999034,0.556025,0.578184,-0.328764,-28.685175,0.764757,4.073237,0.832083,0.764757
2,-0.717038,0.365559,-0.927742,82.991934,0.810689,1.144460,-0.012100,0.954194,1.144460
3,0.225307,0.967796,0.945869,45.613564,-70.239635,13.792260,0.002383,0.970107,13.792260
4,0.162665,0.988917,-0.920230,2.021206,24.912572,10.053899,0.032640,0.767094,10.053899
...,...,...,...,...,...,...,...,...,...
95,-0.501878,-0.473851,-0.140780,17.210673,-35.857416,0.628002,-0.030011,0.766504,0.628002
96,-0.994162,0.974079,0.799942,3.588914,-11.950696,1.975507,-0.279850,0.958758,1.975507
97,-0.435053,0.216801,-0.999299,286.548594,-65.244882,-1.347052,-0.002151,0.987458,-1.347052
98,-0.970093,0.733418,-0.274070,-0.949541,7.441638,-2.677261,1.174980,0.669198,-2.677261


In [ ]:
model_2 = createTableDOM(functions, count, 2, np.array(model_1))
model_2

,x1(2),x2(2),x3(2),v1(2),v2(2),v3(2),y1(2),y2(2),y3(2)
0,0.867069,0.107692,-0.462690,0.182547,-93.800062,1.888612,13.477672,0.974639,1.888612
1,0.513823,0.471804,-0.714355,-4.297545,-63.337056,1.575097,-0.124773,0.999995,1.575097
2,0.899070,0.035050,-0.095124,0.672006,89.789424,1.848675,6.775977,0.980632,1.848675
3,-0.292009,-0.331804,-0.463483,11.917993,-15.372396,4.514738,-0.022985,0.990217,4.514738
4,0.399357,0.814504,-0.159391,7.943627,29.512668,1.271569,0.035203,0.977530,1.271569
...,...,...,...,...,...,...,...,...,...
95,-0.994000,-0.959673,-0.892225,2.217823,-30.596258,9.581680,-0.456133,0.395296,9.581680
96,-0.480784,0.484173,-0.108917,3.633702,84.634352,14.472177,-0.131849,0.971806,14.472177
97,0.876784,-0.899292,0.808253,0.702501,10.882018,0.191935,1.232372,0.436759,0.191935
98,0.388547,0.839755,-0.361315,0.266073,-84.634912,1.624682,0.993319,0.999274,1.624682
